# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Load the csv exported in Part I to a DataFrame
city_file_location = os.path.join("..","WeatherPy","output_data","city_weather_data.csv")
city_df = pd.read_csv(city_file_location)
city_df.head()

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,kapaa,22.0752,-159.3190,71.6,94.0,90.0,23.02,US,1.611112e+09
1,haines junction,60.7522,-137.5108,21.2,93.0,83.0,2.30,CA,1.611112e+09
2,punta arenas,-53.1500,-70.9167,59.0,58.0,75.0,16.11,CL,1.611111e+09
3,east london,-33.0153,27.9116,69.3,77.0,60.0,13.69,ZA,1.611112e+09
4,ushuaia,-54.8000,-68.3000,60.8,51.0,40.0,24.16,AR,1.611111e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Access maps with unique API key
gmaps.configure(g_key)

In [11]:

#Set parameters for gmap
locations = city_df[["Lat", "lng"]]
humidity = city_df["Humidity"].astype(float)

#Plot the heatmap
figure = gmaps.figure()

#create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=50,point_radius=1.5)

#add the layer to the figure
figure.add_layer(heat_layer)
4

#display figure
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
#create a new df to allow for weather criteria. Max Temp lower than 80 bu thigher than 70
#wind speed less than 10 mph, zero cloudiness

conditions_df = city_df.copy()
conditions_df = conditions_df.loc[conditions_df["Max Temp"] >=70]
conditions_df = conditions_df.loc[conditions_df["Max Temp"] <=80]
conditions_df = conditions_df.loc[conditions_df["Wind Speed"] <=10]
conditions_df = conditions_df.loc[conditions_df["Cloudiness"] ==0]
conditions_df = conditions_df.dropna()
conditions_df = conditions_df.reset_index(drop=True)
conditions_df




,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,atuona,-9.8000,-139.0333,78.37,70.0,0.0,9.13,PF,1.611112e+09
1,bouna,9.2667,-3.0000,70.93,22.0,0.0,5.23,CI,1.611112e+09
2,new norfolk,-42.7826,147.0587,75.99,24.0,0.0,1.99,AU,1.611112e+09
3,devarkonda,16.7000,78.9333,73.15,55.0,0.0,4.23,IN,1.611112e+09
4,boa vista,2.8197,-60.6733,78.80,65.0,0.0,5.75,BR,1.611112e+09
5,shirpur,21.3500,74.8833,73.08,29.0,0.0,5.01,IN,1.611112e+09
6,paraguari,-25.6208,-57.1472,75.20,69.0,0.0,5.75,PY,1.611112e+09
7,shirdi,19.7667,74.4833,72.03,53.0,0.0,5.12,IN,1.611112e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
#create a dataframe with the columns that will be used for the project / add an additonal column for the hotel names
hotel_df = pd.DataFrame(conditions_df, columns=["City", "Country", "Lat", "lng"])

#empty column to place hotel names
hotel_df["Hotel Name"]= ""

hotel_df

,City,Country,Lat,lng,Hotel Name
0,atuona,PF,-9.8000,-139.0333,
1,bouna,CI,9.2667,-3.0000,
2,new norfolk,AU,-42.7826,147.0587,
3,devarkonda,IN,16.7000,78.9333,
4,boa vista,BR,2.8197,-60.6733,
5,shirpur,IN,21.3500,74.8833,
6,paraguari,PY,-25.6208,-57.1472,
7,shirdi,IN,19.7667,74.4833,


In [15]:
#creates a list to capture hotel_names to insert into dataframe
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)
        
#apend dataframe with hotel names        
hotel_df['Hotel Name'] = hotel_name
hotel_df

,City,Country,Lat,lng,Hotel Name
0,atuona,PF,-9.8000,-139.0333,Hôtel Hiva Oa Hanakee Lodge
1,bouna,CI,9.2667,-3.0000,GNIME'S HÔTEL
2,new norfolk,AU,-42.7826,147.0587,Tynwald Willow Bend Estate
3,devarkonda,IN,16.7000,78.9333,Vishnu Priya Lodge
4,boa vista,BR,2.8197,-60.6733,Hotel Orla do Rio Branco
5,shirpur,IN,21.3500,74.8833,Hotel AARAM
6,paraguari,PY,-25.6208,-57.1472,Cerro Jere/ Mbatovi
7,shirdi,IN,19.7667,74.4833,Hotel Sai Palace Express


In [16]:
#clean the data by dropping empty rows or those with data missing
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Lat,lng,Hotel Name
0,atuona,PF,-9.8000,-139.0333,Hôtel Hiva Oa Hanakee Lodge
1,bouna,CI,9.2667,-3.0000,GNIME'S HÔTEL
2,new norfolk,AU,-42.7826,147.0587,Tynwald Willow Bend Estate
3,devarkonda,IN,16.7000,78.9333,Vishnu Priya Lodge
4,boa vista,BR,2.8197,-60.6733,Hotel Orla do Rio Branco
5,shirpur,IN,21.3500,74.8833,Hotel AARAM
6,paraguari,PY,-25.6208,-57.1472,Cerro Jere/ Mbatovi
7,shirdi,IN,19.7667,74.4833,Hotel Sai Palace Express


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "lng"]]

In [18]:
# Add marker layer ontop of heat map
# Display figure

fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


fig



Figure(layout=FigureLayout(height='420px'))